In [ ]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import pickle
import random
import numpy as np

In [3]:
historical_transactions = pd.read_csv("dataset/historical_transactions.csv")

In [4]:
history = {}

In [5]:
pbar = tqdm(total=len(historical_transactions))
for index, row in historical_transactions.iterrows():
    key = (row['card_id'], row['merchant_id'])
    if key not in history:
        history[key] = 0
    history[key] += 1
    pbar.update(1)
pbar.close()

In [7]:
pickle.dump(history, open('history.pickle', 'wb'))

KeyboardInterrupt: 

In [9]:
data = [(d[0], d[1], history[d]) for d in history]
random.shuffle(data)
train, test = np.array(data[:-1000]), np.array(data[-1000:])

In [ ]:
dim, lam, lr = 20, 7, 0.01
item_count, user_count = {}, {}
for u, i, r in train:
    user_count[u] = user_count[u]+1 if u in user_count else 1
    item_count[i] = item_count[i]+1 if i in item_count else 1
item_set, user_set = set(item_count.keys()), set(user_count.keys())
prev_MSE, MSE = 40, 39
gamma_i = {i:np.random.rand(dim) for i in item_set}
gamma_u = {i:np.random.rand(dim) for i in user_set}
print 'Start Training'
while prev_MSE-MSE > 10**(-6):
    gamma_i_, gamma_u_ = dict(gamma_i), dict(gamma_u)
    prev_MSE = MSE
    dgamma_i = {i:np.array([2*lam*k for k in gamma_i[i]]) for i in item_set}
    dgamma_u = {i:np.array([2*lam*k for k in gamma_u[i]]) for i in user_set}
    for u, i, r in train:
        prediction = sum(gamma_i[i]*gamma_u[u])
        err = prediction - float(r)
        dgamma_i[i] += 2*err*gamma_u[u]
        dgamma_u[u] += 2*err*gamma_i[i]
    gamma_i = {i:gamma_i[i]-lr*dgamma_i[i]/item_count[i] for i in item_set}
    gamma_u = {i:gamma_u[i]-lr*dgamma_u[i]/user_count[i] for i in user_set}
    MSE = np.mean([(float(r)-sum(gamma_i[i]*gamma_u[u]))**2 for u, i, r in test if u in user_set and i in item_set])
    print 'Validation RMSE: {}'.format(np.sqrt(MSE))

Start Training
Validation MSE: 22.8368499978
Validation MSE: 21.5583667378
Validation MSE: 20.5486812059
Validation MSE: 19.7383868826
Validation MSE: 19.0799470945
Validation MSE: 18.5400404513
Validation MSE: 18.0947810351
Validation MSE: 17.7265716768


In [ ]:
pickle.dump(gamma_u_, open('user_embedding.pickle', 'wb'))

# Training

In [53]:
def feat(i):
    return list(gamma_u_[i['card_id']]) + [i['feature_1'], i['feature_2'], i['feature_3'], 1]
    return [i['feature_1'], i['feature_2'], i['feature_3'], 1]

In [25]:
trainset = pd.read_csv('dataset/train.csv')

In [54]:
X = np.array([ feat(i) for i in trainset.to_dict(orient='records') ])
y = [i['target'] for i in trainset.to_dict(orient='records')]
theta,residuals,rank,s = np.linalg.lstsq(X, y)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
RMSE = np.sqrt(((y-np.matmul(theta,X.T))**2).mean())
print 'Training MSE: ', RMSE

Training MSE:  3.849638217265452


In [56]:
testset = pd.read_csv('dataset/test.csv')
X = np.array([ feat(i) for i in testset.to_dict(orient='records') ])
pred = np.matmul(theta,X.T)
df_sub = pd.DataFrame({"card_id":testset["card_id"].values})
df_sub["target"] = pred
df_sub.to_csv("submission.csv", index=False)